In [1]:
import os
from constants import *

os.environ["PPLX_API_KEY"] = PPLX_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from constants import *
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.chains import OpenAIModerationChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.document import Document

import pandas as pd
import numpy as np

In [3]:
import re
path = "data/reddit_opinion_climate_change.csv"
df = pd.read_csv(path)
df

,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,...,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
0,l8ai9g6,1,A few other things that are left out of the de...,climatechange,2024-06-12 16:36:53,1de0acd,Melodic-Hippo5536,0,1,0,...,71.0,1478.0,1549.0,97,NaN,New report refutes 33 false claims about solar...,0.99,97,0,2024-06-12 06:50:30
1,l8ai96k,1,So your take is not that carbon pricing doesn'...,canada,2024-06-12 16:36:50,1ddqrmw,The_Eternal_Void,0,1,0,...,1579.0,15065.0,16972.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
2,l8ai0r3,1,"Yes, except it's averaging higher.",canada,2024-06-12 16:35:32,1ddqrmw,LeeStrange,0,1,0,...,504.0,5127.0,5646.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
3,l8ai09f,1,"True, it's actually been way more devastating ...",canada,2024-06-12 16:35:28,1ddqrmw,Best-Hotel-1984,0,1,0,...,9.0,4419.0,4428.0,39,NaN,"Expect above-average summer temperatures, Cana...",0.67,39,0,2024-06-11 22:14:35
4,l8ahv26,1,The rich looking to get richer. Go green means...,climateskeptics,2024-06-12 16:34:39,1deajz2,vacouple3,0,1,0,...,352.0,4928.0,5342.0,3,NaN,New Report Reveals Massive Scale of Green Bill...,1.00,3,0,2024-06-12 16:15:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542498,dzyrmqu,2,"Yes, I remember the hype that ended in nothing...",climate_science,2018-06-01 22:18:21,8ntlug,MissPinga,0,2,0,...,301.0,3516.0,3817.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542499,dzyf2d7,2,There were a [lot of people who lost a lot of ...,climate_science,2018-06-01 18:58:56,8ntlug,silence7,0,2,0,...,982947.0,249288.0,1256803.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542500,dzyesw9,1,"Well, it can be used as a bio fuel. This is an...",climate_science,2018-06-01 18:55:08,8ntlug,dano1066,0,1,0,...,12834.0,184926.0,198366.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16
542501,dzybxzq,5,There's no money in algae,climate_science,2018-06-01 18:13:20,8ntlug,ZippymcOswald,0,5,0,...,39918.0,26950.0,67462.0,19,If algae is responsible for cleaning our air m...,Why dont we farm algae on a large scale to red...,0.96,19,0,2018-06-01 17:09:16


In [40]:
df["self_text"].sample(10000, random_state=42)

170934    First, allow me to commend you for your courag...
525301    You're sidestepping.\n\n&gt;I'd say we need to...
366508    The issue with high altitudes is they almost a...
1305      Clouds are parameterized in GCMs. Ditto on oce...
513025    The pod fellow, not Dr. Peterson, though I am ...
                                ...                        
449981                                  IMF says otherwise.
179607    &gt; is still *paid* at max\n\nFTFY.\n\nAlthou...
445002    it's so funny that people don't recognize this...
518782      That's living in fear, and is not a good reason
357499                The holy trinity of Coal Oil and Gas.
Name: self_text, Length: 10000, dtype: object

In [41]:
def clean_tweet(text):
    # keep handles for model
    #text = re.sub("@\w+", "", text) # Remove handle   

    text = re.sub("[^\x20-\x7E]", "", text)
    text = re.sub('http\S+', "", text)

    return text

documents = df['self_text'].sample(1000, random_state=42)\
    .apply(clean_tweet)\
    .apply(Document)\
    .to_list()
documents

[Document(page_content="First, allow me to commend you for your courageous level of optimism. That said, I must agree that this is a naive take on this issue. I think that anyone that is legally an adult in our society is personally responsible for what they choose to believe or what information sources take as credible. Further, parents are personally responsible for taking care of their children and looking out for their welfare and best interests. There's room to debate a great many issues in our society, but not getting one's children vaccinated against Measles. It is dangerous and negligent, and makes less sense than the flat earth movement. At least the flat earth people are only silly."),
 Document(page_content="You're sidestepping.&gt;I'd say we need to overcome our meekness and do something about those that are fuelling this problem and making it needlessly worse every day. **It would seem asking them nicely has not worked**&amp;#x200B;&gt;Meanwhile, the flames are getting hot

In [46]:
if "vectorstore" in globals():
    vectorstore.delete_collection()
vectorstore = Chroma.from_documents(documents, OpenAIEmbeddings())

In [43]:
pplx_llm = ChatPerplexity(temperature=0, model="llama-3-70b-instruct")
openai_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
moderate = OpenAIModerationChain()

In [59]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

retriever = vectorstore.as_retriever(search_kwargs={"k": 20})


def format_docs(docs):
    return "\n\n".join(f"{i+1}. {doc.page_content}" for i,doc in enumerate(docs))

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Do not make any assumptions. "
    "Keep the answer descriptive and consise. "
    "\nQuestion: {question} "
    "\nContext: {context}  "
    "\nAnswer: "
)

prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | openai_llm
    | StrOutputParser()
)

In [60]:
query = "What are the overall opinion on ocean acidification?"

retrieved_docs = retriever.invoke(query)
print(format_docs(retrieved_docs))
print("-"*20)
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

1. How aware are climate scientists of ocean acidification and the feedback loops it will cause

2. per Doney (2009), ocean ph has dropped from \~8.10 to \~8.08 (a \~5% drop on the logarithmic pH scale) in the past \~20 yearsinstrumentally insignificant    Doney, S. C., V. J. Fabry, R. A. Feely, and J. Kleypas. 2009. Ocean Acidification: The Other CO2 Problem. Annual Review of Marine Sciences 1 :169-192.      Per: "Precision and accuracy of spectrophotometric pH measurements at environmental conditions in the Baltic Sea," Science, 2014:    ...Using a...spectrophotometric measurement system... an accuracy between 0.01 and 0.02 was achieved within the observed pH..."

3. Climate change is a constant force in coral development. Corals first appeared in the Cambrian, then became a dominant fossil in the Ordovician when global conditions were much warmer than today, and co2 in the thousands of ppm (so much for acidification). They survived all of the extinctions, including both the Permo-Tr